In [1]:
import pandas as pd
import numpy as np

In [2]:
import glob

all_dfs = []

for one_filename in glob.glob('../data/*,*.csv'):
    print(f'Loading {one_filename}...')
    city, state = one_filename.removeprefix('../data/').removesuffix('.csv').split(',')
    one_df = pd.read_csv(one_filename, 
                     usecols=[1, 2, 19],
                     names=['max_temp', 'min_temp', 'precipMM'],
                     header=0)
    one_df['city'] = city.replace('+', ' ').title()
    one_df['state'] = state.upper()
    all_dfs.append(one_df)

df = pd.concat(all_dfs)
print(df.head())
print(df.info())

Loading ../data/san+francisco,ca.csv...
Loading ../data/new+york,ny.csv...
Loading ../data/springfield,ma.csv...
Loading ../data/boston,ma.csv...
Loading ../data/springfield,il.csv...
Loading ../data/albany,ny.csv...
Loading ../data/los+angeles,ca.csv...
Loading ../data/chicago,il.csv...
   max_temp  min_temp  precipMM           city state
0        13         8       0.0  San Francisco    CA
1        13         8       0.0  San Francisco    CA
2        13         8       0.0  San Francisco    CA
3        13         8       0.0  San Francisco    CA
4        13         8       0.0  San Francisco    CA
<class 'pandas.core.frame.DataFrame'>
Index: 5824 entries, 0 to 727
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   max_temp  5824 non-null   int64  
 1   min_temp  5824 non-null   int64  
 2   precipMM  5824 non-null   float64
 3   city      5824 non-null   object 
 4   state     5824 non-null   object 
dtypes: float64(1), 

In [3]:
def has_multiple_readings_at_least(mini_df):
    return mini_df.loc[
        mini_df['precipMM'] >= 15,
        'precipMM'].count() >= 3

In [4]:
df.groupby(['city', 'state']).filter(has_multiple_readings_at_least)[['city', 'state']].drop_duplicates()

,city,state
0,New York,NY
0,Boston,MA
0,Los Angeles,CA


In [5]:
def has_multiple_readings_at_least(mini_df, min_mm, times):
    return mini_df.loc[
        mini_df['precipMM'] >= min_mm,
        'precipMM'].count() >= times



(
    df
    .groupby(['city', 'state'])
    .filter(has_multiple_readings_at_least,
            min_mm = 10,
            times = 3)
    [['city', 'state', 'precipMM']]
    .drop_duplicates()
)

,city,state,precipMM
0,San Francisco,CA,0.0
30,San Francisco,CA,0.8
34,San Francisco,CA,0.4
46,San Francisco,CA,4.0
48,San Francisco,CA,0.3
...,...,...,...
633,Los Angeles,CA,1.2
652,Los Angeles,CA,7.6
683,Los Angeles,CA,14.7
684,Los Angeles,CA,14.3


In [6]:
def has_multiple_readings_at_least(mini_df, min_mm, times):
    return mini_df.loc[
        ((mini_df['precipMM'] >= min_mm) &
        (mini_df['min_temp'] <= 0)), 
        'precipMM'].count() >= times


(
    df
    .groupby(['city', 'state'])
    .filter(has_multiple_readings_at_least,
            min_mm = 10,
            times = 3)
    [['city', 'state']]
    .drop_duplicates()
)

,city,state
0,New York,NY
0,Boston,MA
0,Albany,NY


In [ ]:
import glob

all_dfs = []

for one_filename in glob.glob('../data/*,*.csv'):
    print(f'Loading {one_filename}...')
    city, state = one_filename.removeprefix('../data/').removesuffix('.csv').split(',')
    one_df = pd.read_csv(one_filename,
                    header=0)
    one_df['city'] = city.replace('+', ' ').title()
    one_df['state'] = state.upper()
    all_dfs.append(one_df)

df_ = pd.concat(all_dfs, axis=1)


Loading ../data/san+francisco,ca.csv...
Loading ../data/new+york,ny.csv...
Loading ../data/springfield,ma.csv...
Loading ../data/boston,ma.csv...
Loading ../data/springfield,il.csv...
Loading ../data/albany,ny.csv...
Loading ../data/los+angeles,ca.csv...
Loading ../data/chicago,il.csv...


In [11]:
df_.to_csv('weather_regular_heading.csv', sep= ',' , header= True)
# dont like to make that datafreme every time. So
# why not make a csv with all the data available. 
# If I need to work with it again, I will load the csv file only. 

In [9]:
def proportion_of_city_precip(s):
    return s / s.sum()


df['precip_pct'] = df.groupby('city')['precipMM'].transform(proportion_of_city_precip)
df.head()

,max_temp,min_temp,precipMM,city,state,precip_pct
0,13,8,0.0,San Francisco,CA,0.0
1,13,8,0.0,San Francisco,CA,0.0
2,13,8,0.0,San Francisco,CA,0.0
3,13,8,0.0,San Francisco,CA,0.0
4,13,8,0.0,San Francisco,CA,0.0


In [10]:
df.groupby(['city', 'state'])['precip_pct'].max()

city           state
Albany         NY       0.029228
Boston         MA       0.048302
Chicago        IL       0.057257
Los Angeles    CA       0.059242
New York       NY       0.055149
San Francisco  CA       0.056509
Springfield    IL       0.030977
               MA       0.023459
Name: precip_pct, dtype: float64